In [22]:
import numpy as np
import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt, style
style.use('ggplot')

import json

In [23]:
train = open('train.json','r').read()
train_json = json.loads(train)
test = open('test.json','r').read()
test_json = json.loads(test)
train_json[1]

{'cuisine': 'southern_us',
 'id': 25693,
 'ingredients': ['plain flour',
  'ground pepper',
  'salt',
  'tomatoes',
  'ground black pepper',
  'thyme',
  'eggs',
  'green tomatoes',
  'yellow corn meal',
  'milk',
  'vegetable oil']}

In [24]:
df_dict = dict()
df_dict['id']=[]
df_dict['cuisine']=[]
df_dict['ingredient_list'] = []
train_df = pd.DataFrame(df_dict,dtype=np.int64)
test_df = train_df.copy()
train_df.head()

,id,cuisine,ingredient_list


In [25]:
import re
i = 0
for curr_json in train_json:    
    ingredient_list = " ".join([ re.sub('\s',"_",ingredient) for ingredient in curr_json['ingredients'] ])
    train_df.loc[i] =  [curr_json['id'],curr_json['cuisine'],ingredient_list]
    i+=1
train_df.head()

,id,cuisine,ingredient_list
0,10259,greek,romaine_lettuce black_olives grape_tomatoes ga...
1,25693,southern_us,plain_flour ground_pepper salt tomatoes ground...
2,20130,filipino,eggs pepper salt mayonaise cooking_oil green_c...
3,22213,indian,water vegetable_oil wheat salt
4,13162,indian,black_pepper shallots cornflour cayenne_pepper...


In [26]:
test_df.drop(['cuisine'],1,inplace=True)
i = 0
for curr_json in test_json:    
    ingredient_list = " ".join([ re.sub('\s',"_",ingredient) for ingredient in curr_json['ingredients'] ])
    test_df.loc[i] =  [curr_json['id'],ingredient_list]
    i+=1
test_df.head()

,id,ingredient_list
0,18009,baking_powder eggs all-purpose_flour raisins m...
1,28583,sugar egg_yolks corn_starch cream_of_tartar ba...
2,41580,sausage_links fennel_bulb fronds olive_oil cub...
3,29752,meat_cuts file_powder smoked_sausage okra shri...
4,35687,ground_black_pepper salt sausage_casings leeks...


In [27]:
Y_train_all = train_df['cuisine']
train_df.drop('cuisine',1,inplace=True)
print(Y_train_all.head())

0          greek
1    southern_us
2       filipino
3         indian
4         indian
Name: cuisine, dtype: object


In [28]:
print(train_df.shape,test_df.shape)
X_all = train_df.append(test_df)
print(X_all.shape)
from sklearn.feature_extraction.text import CountVectorizer
ingredient_list = X_all['ingredient_list']
cv = CountVectorizer().fit(ingredient_list)
ingredient_list_count = cv.transform(ingredient_list)
print(ingredient_list_count.shape)

(39774, 2) (9944, 2)
(49718, 2)
(49718, 7299)


In [29]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(ingredient_list_count)
ingredient_list_tfidf = tfidf_transformer.transform(ingredient_list_count)
ingredient_list_tfidf.shape

(49718, 7299)

In [31]:
X_train_all = ingredient_list_tfidf[:39774]
X_test = ingredient_list_tfidf[39774:]
from sklearn.model_selection import train_test_split
X_train,X_validation,Y_train,Y_validation = train_test_split(X_train_all,Y_train_all,test_size=0.2)
X_test_index = test_df['id']
print("train :{}{}, test: {} index: {}, validation: {}{}".format(X_train.shape,Y_train.shape,
                                                                X_test.shape,X_test_index.shape,
                                                                X_validation.shape,Y_validation.shape))

train :(31819, 7299)(31819,), test: (9944, 7299) index: (9944,), validation: (7955, 7299)(7955,)


In [33]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,Y_train)
clf.score(X_validation,Y_validation)

0.64512884978001261

In [36]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(X_train,Y_train)
clf.score(X_validation,Y_validation)

0.67177875549968569

In [35]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
clf.score(X_validation,Y_validation)

0.65405405405405403

In [37]:
predictions = clf.predict(X_test)
print(predictions.shape)

(9944,)


In [39]:
res = pd.DataFrame({
    "id":X_test_index,
    "cuisine":predictions
})

In [41]:
res.to_csv("sol.csv",header=True,index=False)